## Web Scraping(From Amazon)
## My scraper extract the data from Amazon
## The user have to just enter the item and number of pages from which he/she want to extract data.


## Importing libraries

In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver

import pandas as pd

from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager


In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
There is no [win32] chromedriver for browser 106.0.5249 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\PC\.wdm\drivers\chromedriver\win32\106.0.5249.61]
C:\Users\PC\AppData\Local\Temp\ipykernel_2236\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Getting Urls

In [3]:
def get_url(item_search,pages):
    url_list = []
    url = 'https://www.amazon.com/s?k={}&page='
    item_search = item_search.strip()
    item_search = item_search.replace(' ','+')
    for page_num in range(pages):
        url_list.append(url.format(item_search)+str(page_num+1))

    return url_list

## Scraper

In [4]:
def scraper(item_name,pages):
    data=[]
    urls=get_url(item_name,pages) #Enter item and number of pages
    
    for url in urls:  # urls of different pages
        driver.get(url) # driver of first page
        soup=BeautifulSoup(driver.page_source,'html.parser') # soup for different pages
        results = soup.find_all("div",{"data-component-type":"s-search-result"}) # content of different pages
        for item in range(len(results)): # looping through a page to get all items
            dic={}
            
            
            item_url='https://www.amazon.com'+results[item].a.get('href') # url of an item page
            driver.get(item_url) #driver for an item contents
            soup1=BeautifulSoup(driver.page_source,'html.parser') # soup for an item page
            item_specs_heading = soup1.find_all('th',{'class':'a-color-secondary a-size-base prodDetSectionEntry'})
            item_specs = soup1.find_all('td',{'class':'a-size-base prodDetAttrValue'}) # getting content of item page

            for num in range(len(item_specs)):
                try:
                    dic[item_specs_heading[num].text.strip()] = item_specs[num].text.strip().replace('\u200e','')
                except:
                    dic[item_specs_heading[num].text.strip()] = None

                
            try:
                dic['Price'] = soup1.find('span',{'id':'priceblock_ourprice'}).text
            except:
                dic['Price']=None
            
            try:
                dic["Ratings"] = soup1.find('span',{'id':'acrCustomerReviewText'}).text
            except:
                dic["Ratings"] = None
            
            try:
                dic["Reviews"] = soup1.find('span',{'class':'a-icon-alt'}).text
            except:
                dic["Reviews"] = None
                              
            try:
                dic["Reviews Link"] = 'https://www.amazon.com'+soup1.find('a',{'class':'a-link-normal'}).get('href')
            except:
                dic["Reviews Link"] = None
                
            dic = list(dic.items())
            dic.sort()
            dic = dict(dic)
            data.append(dic)
    driver.close()
    df=pd.DataFrame(data)
    print(df)
    df.to_csv('AmazonLaptopExtractedData.csv')

In [5]:
scraper('laptop',1)

          ASIN Average Battery Life (in hours)    Brand Card Description  \
0   B09KMFJQX6                        12 Hours     Acer     UHD Graphics   
1   B09RNC69K8                             NaN   Lenovo       Integrated   
2   B0B3L3YB4N                             NaN     Acer       Integrated   
3   B09HVC79PC                        10 Hours     Acer       Integrated   
4   B08D92T45H                             NaN     ASUS       Integrated   
5   B08B6F1NNR                         7 Hours   Lenovo       Integrated   
6   B09J8V32X4                             NaN  SAMSUNG        Dedicated   
7   B09YRY6QCX                             NaN     ASUS       Integrated   
8   B08NTP5RTN                        10 Hours     Acer       Integrated   
9   B0B9YRLB3D                             NaN   Lenovo       Integrated   
10  B09VZR2BGN                             NaN       HP       Integrated   
11  B09R8YJB9W                             NaN  SAMSUNG       Integrated   
12  B0B9KDJJ